# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
torch.__version__

'1.4.0'

In [3]:
df = pd.read_csv('smile-annotations-final.csv', names=['id','text','category'])
df.set_index('id', inplace=True)

In [4]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [5]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
df = df[~df.category.str.contains('\|')]

In [7]:
df = df[df.category != 'nocode']

In [8]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
possible_labels = df.category.unique()

In [10]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [11]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [12]:
df['label'] = df.category.replace(label_dict)
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Task 3: Training/Validation Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)

In [15]:
df['data_type'] = ['not_set'] * df.shape[0]

In [16]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [17]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [18]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [19]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [21]:
!pip install jupyter
!pip install ipywidgets==7.4.2
!jupyter nbextension enable --py widgetsnbextension
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple


Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_legnth=256,
    return_tensor='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_legnth=256,
    return_tensor='pt'
)

input_ids_train = torch.tensor(encoded_data_train['input_ids'])
attention_masks_train = torch.tensor(encoded_data_train['attention_mask'])
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = torch.tensor(encoded_data_val['input_ids'])
attention_masks_val = torch.tensor(encoded_data_val['attention_mask'])
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

...

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.
Keyword arguments {'max_legnth': 256, 'return_tensor': 'pt'} not recognized.

In [25]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)
dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val,
                              labels_val)

In [26]:
len(dataset_train)

1258

In [27]:
len(dataset_val)

223

## Task 5: Setting up BERT Pretrained Model

In [28]:
from transformers import BertForSequenceClassification

In [29]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                      num_labels=len(label_dict),
                                      output_attentions=False,
                                      output_hidden_states=False
                                     )

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [31]:
batch_size = 4 #32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## Task 7: Setting Up Optimizer and Scheduler

In [32]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [33]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ImportError: torch>=1.5.0 is required for a normal functioning of this module, but found torch==1.4.0.

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataset_train) * epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [34]:
#preds = [0.9, 0.05, 0.05, 0, 0, 0]
#preds = [1, 0, 0, 0, 0, 0]

In [35]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    lables_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [36]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        y_preds =  preds_flat[labels_flat==label]
        y_true  = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [37]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

NameError: name 'np' is not defined

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [39]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                       leave=False,
                       disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask':batch[1],
            'labels': batch[2]
        }
        outputs = model(**inputs)
    
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss' : '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total / len(dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')

## Task 10: Loading and Evaluating our Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model.to(device)
pass

In [ ]:
model.load_state_dict(
    torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
              map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)